In [ ]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
from numba import cuda

In [ ]:
@cuda.jit(device=True)
def binary_search(array, value):
    left, right = 0, array.shape[0] - 1
    while left <= right:
        mid = (left + right) // 2
        if array[mid] == value:
            return True
        elif array[mid] < value:
            left = mid + 1
        else:
            right = mid - 1
    return False

In [ ]:
@cuda.jit
def mnn_kernel(dist_list, mnn_score_gpu):
    thread_id = cuda.grid(1)
    if thread_id < dist_list.shape[0]:
        current_neighbors = dist_list[thread_id]
        mutual_score = 0
        for neighbor_id in current_neighbors:
            neighbors_of_neighbor = dist_list[neighbor_id]
            if binary_search(neighbors_of_neighbor, thread_id):
                mutual_score += 1
        mnn_score_gpu[thread_id] = mutual_score

In [ ]:
class Knn_utils:
    def __init__(self, row, col, num_neighbors):
        self.row = row
        self.col = col
        self.num_neighbors = num_neighbors
        self.matrix = None
        self.dist_list = None

    def create_matrix(self):
        self.matrix = np.random.randint(1, 21, size=(self.row, self.col), dtype=np.int32)
        for row in self.matrix:
            print(' '.join(map(str, row)))

    def compute_neighbors(self):
        knn_model = NearestNeighbors(n_neighbors=self.num_neighbors + 1, algorithm='auto').fit(self.matrix)
        _, self.dist_list = knn_model.kneighbors(self.matrix)
        self.dist_list = np.ascontiguousarray(self.dist_list[:, 1:])
        for i in range(self.dist_list.shape[0]):
            self.dist_list[i].sort()

    def calculate_mnn_scores(self):
        dist_list_gpu = cuda.to_device(self.dist_list)
        mnn_scores = np.zeros(self.row, dtype=np.int32)
        mnn_scores_gpu = cuda.to_device(mnn_scores)
        threads_per_block = 256
        blocks_per_grid = (self.row+threads_per_block -1) // threads_per_block
        mnn_kernel[blocks_per_grid, threads_per_block](dist_list_gpu, mnn_scores_gpu)
        return mnn_scores_gpu.copy_to_host()

In [ ]:

if __name__ == "__main__":
    mnn_calculator = Knn_utils(row=6, col=6, num_neighbors=3)
    mnn_calculator.create_matrix()
    mnn_calculator.compute_neighbors()
    mnn_scores = mnn_calculator.calculate_mnn_scores()
    print("MNN scores for all points:")
    for i, score in enumerate(mnn_scores):
        print(f"Point {i}: {score}")

20 12 2 8 15 16
19 1 12 4 14 18
15 16 17 13 12 4
4 14 11 15 12 19
17 9 1 4 1 12
9 19 7 10 11 9


/usr/local/lib/python3.11/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


MNN scores for all points:
Point 0: 3
Point 1: 2
Point 2: 2
Point 3: 2
Point 4: 2
Point 5: 3
